### import packages

In [1]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity

### Specify directories to data files

In [2]:
lb_dir = "/LHA3_R3_medium/segmentation/LHA3_R3_medium-c2.tif"
bad_roi='/test/bad_roi_list.npy'
spot_dir = '/test/LHA3_R3_medium/spots/'
out_dir = '/test/LHA3_R3_medium/spots/'
img_dir='/test/LHA3_R3_medium/stitching/export.n5'

### Load segmentation mask

In [3]:
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

number of ROIs in total: 2113
number of bad ROIs: 539


### Extract ROI properties
The roi_prop function exports the following: 
- ROI ID
- position (x, y, z in um, pre-expansion)
- size (volume, pre-expansion) 
- mionr_axis_length
- major_axis_length
- eccentricity =  mionr_axis_length/major_axis_length
- solidity = ROI volume/ROI convex hull

In [4]:
df=roi_prop.roi_prop(lb)

### Remove bad ROIs and save the file

In [15]:
df_filtered=df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered.roi=df_filtered.roi.astype(int)
df_filtered.to_csv(out_dir+'roi.csv')

### Lipofuscin subtraction (optional)

In [7]:
# Load spot files for lipofuscin subtraction
channel_1=np.loadtxt(spot_dir+'/'+'merged_points_c0.txt',delimiter=',')
channel_2=np.loadtxt(spot_dir+'/'+'merged_points_c1.txt',delimiter=',')
# Identify lipofuscin spots
c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# Remove lipofuscin spots
true_pos_c0 = np.delete(channel_1, pAind, axis=0)
true_pos_c1 = np.delete(channel_2, pBind, axis=0)

### spot extraction and save file

In [10]:
spotcount=spot.spot_counts(lb,spot_dir)
spotcount.to_csv(out_dir+'/spots.csv')

Load: /Volumes/multifish/Pipeline/test/LHA3_R3_medium/spots/merged_points_c0.txt
Load: /Volumes/multifish/Pipeline/test/LHA3_R3_medium/spots/merged_points_c1.txt
Load: /Volumes/multifish/Pipeline/test/LHA3_R3_medium/spots/merged_points_c3.txt


### Intensity measurement and save file

In [4]:
df_intensity=intensity.measure_intensity(lb, img_dir,'c0')
df_intensity.to_csv(out_dir+'/intennsity_c0.csv')

In [5]:
df_intensity

,roi,c0
0,1,216.597071
1,2,213.015920
2,3,257.894697
3,4,218.268243
4,5,215.610488
...,...,...
2108,2109,215.273395
2109,2110,216.327219
2110,2111,214.096334
2111,2112,213.316004
